In [ ]:
# This entire notebook was written by Jerry (`book_details` provided by Everett)

import requests
import re
from bs4 import BeautifulSoup
import string
from time import sleep
from collections import namedtuple
from google.colab import drive
from google.colab import drive
drive.mount('/gdrive')

urls = [
    'https://www.gutenberg.org/cache/epub/9902/pg9902.html', # The Middle of Things
    'https://www.gutenberg.org/cache/epub/9807/pg9807.html', # Scarhaven Keep
    'https://www.gutenberg.org/cache/epub/9834/pg9834.html', # The Talleyrand Maxim
    'https://www.gutenberg.org/cache/epub/10373/pg10373.html', # The Middle Temple Murder
    'http://www.gutenberg.org/cache/epub/12239/pg12239.html' # Dead Men's Money
]

Mounted at /gdrive


In [ ]:
book_details = {
    "The Middle of Things": {
        "Detective": "Viner",
        "Perp": "Cortelyon", # text=Cortelyon saw that by killing Ashton he alone would have the secret; he evidently got two accomplices who were necessary to him
        # Cave (Nugent Starr) and Killenhall, his accomplices
        "Crime": "murder", #chapter 1 last sentence. Text=Then he saw white linen, and a bloodstain slowly spreading over its glossy surface.
        "Other Suspects": "Penkridge",
        "Victim": "Richard Ashton",
        "Characters": ['Viner', 'Pawle', 'Ashton', 'Carless', 'Wickham', 'Hyde', #victim
                       'Marketstoke', 'Ellingham', 'Drillford', 'Killenhall', 
                       'Methley', 'Perkwite', 'Miss Wickham', 'Felpham', 
                       'John Ashton', 'Cave', 'Miss Penkridge', 'Fosdick', 
                       'Millington-Bywater', 'Cortelyon', 'Armitstead', 'Barleyfield', # perp
                       'Summers', 'Van Hoeren', 'Millwaters', 'Woodlesford', 'Penkridge', 
                       'Stephens', 'Portlethwaite', 'Martincole', 'Richard', 'Bigglesforth', 
                       'Driver', 'Nugent Starr', 'Avice', 'Pelver', 'Earl', 'Langton Hyde',
                       'Roscombe', 'Langton', 'Lincoln']
    },
    "The Middle Temple Murder" : {
        "Detective": "Frank Spargo|Frank|Spargo", # "Frank Spargo",
        "Perp": "Jane Baylis|Miss Baylis", # Jane Baylis murdered John Maitland
        "Crime": "murder", # john maitland a bank manager charged with embezzlement. changed his name. had a son named Breton. then was killed at cardlestone/Chamberlayne's
        "Other Suspects": "Stephen Aylmore|Chamberlayne|Myerst",
        "Victim": "John Maitland|John Marbury", # john marbury is maitland he changed his name
        "Characters": ['Spargo', 'Breton', 'Marbury', 'Elphick', 'Aylmore', 'Maitland', 
                       'Rathbury', 'Quarterpage', 'Cardlestone', 'Myerst', 'Baylis', 'John Marbury', # victim
                       'John Maitland', 'Ronald Breton', 'Walters', 'Criedir', 'Gutch', 'Driscoll', # victim
                       'Mother Gutch', 'Webster', 'Anderson', 'Jane Baylis', 'Mollison', 'Jessie Aylmore', #perp
                       'Starkey', 'Jessie', 'Stephen Aylmore', 'Evelyn', 'Cooper', 'Ronald', 'Crowfoot', 'Doolittle', 
                       'Ainsworth', 'David Lyell', 'Vallas', 'Kaye', 'Stephens', 'Septimus Elphick', 'Miss Baylis', 
                       'Robertson', 'Lyell']
    },
    "Scarhaven Keep" : {
        "Detective": "Richard Copplestone|Copplestone", 
        "Perp": "Peter Chatfield|Addie Chatfield|Addie|Martin|Andrius|Martin Andrius", # Martin, Addie Chatfield
        "Crime": "murder|fraud", # plot twist. originally they are investigating the murder of basset oliver. this turns out to be an accident not murder.
        "Other Suspects": "Audrey Greyle",
        "Victim": "Audrey Greyle", 
        "Characters": ['Copplestone', 'Greyle', 'Gilling', 'Chatfield', 'Vickers', 
                       'Audrey', 'Marston Greyle', 'Bassett Oliver', 'Cresswell', 'Oliver', # victim
                       'Stafford', 'Petherton', 'Wooler', 'Addie', 'Dennie', 'Cresswell Oliver', 
                       'Peter Chatfield', 'Spurge', 'Andrius', 'Pike', 'Ewbank', 'Addie Chatfield', # perp
                       'Zachary Spurge', 'Scarhaven', 'Squire', 'Jim', 'Peter', 'Altmore', 'Audrey Greyle', 
                       'Bassett', 'Martin', 'Rothwell', 'Montmorency', 'Hackett', 'Miss Greyle', 'Glen', 'Salmon', 
                       'Jerramy', 'Elkin', 'Hobkin', 'Tretheway', 'Valdey', 'Jim Spurge', 'Valentine Greyle', 'Stephen John']
    }, 
    "The Talleyrand Maxim": {
        "Detective": "Linford Pratt|Pratt", # detective commits suicide at the end of the book
        "Perp": "Harper Mallathorpe|Harper|Mallathorpe",
        "Crime": "murder",  # plot twist initially they investigate a murder, but it turns out the man left town and went to america
        #harper destroyed her husband's will because he left his money to a charity
        "Other Suspects": "Nesta",
        "Victim": "John Mallathorpe",
        "Location": "Barford",
        "Characters": ['Pratt', 'Eldrick', 'Collingwood', 'Mallathorpe', # includes the entire mallathrope family
                       'Nesta', 'Parrawhite', 'Murgatroyd', 'John Mallathorpe', #victim 
                       'Pickard', 'Byner', 'Esther Mawson', 'Bartle',
                       'Antony Bartle', 'Cobcroft', 'Prydale', 
                       'Nesta Mallathorpe', 'Harper', 'Robson', 'Normandale Grange',
                       'Esther', 'Linford Pratt', 'Miss Mallathorpe', 'Harper Mallathorpe', # perp
                       'Gaukrodger', 'James Parrawhite', 'Marshall', 'Barford', 'Pascoe', 
                       'Jabey Naylor', 'Parsons', 'Clough', 'Stringer', 
                       'Shepherd', 'Bartle Collingwood']
    },
    "Dead Men's Money": {
        "Detective": "Hugh Moneylaw|Hugh|Moneylaw", # hugh is writing the story in first person
        "Perp": "Gilbert Carstairs|Meekin", # Meek
        "Crime": "murder", # Meekin impersonates Gilbert Carstairs and steals his money. 
        # a few people find out about it and want to be paid off. meekin kills them
        "Other Suspects": "",
        "Victim": "Crone",
        "Characters": ['Lindsey', 'Gilbert', 'Portlethorpe',
                       'Chisholm', 'Gilverthwaite', 'Gilbert Carstairs',
                       'Crone', 'Phillips', 'Maisie', 'Smeaton', 'Murray', #victim
                       'Hugh', 'Hollins', 'Moneylaws', 'Michael Carstairs',
                       'Carstairs', 'Ridley', 'Ralston', 'Gavin Smeaton', 
                       'John Phillips', 'James Gilverthwaite', 'Lady Carstairs',
                       'Peebles', 'Elphinstone', 'Andrew Dunlop', 'Abel Crone',
                       'Michael', 'Paley', 'Hanson', 'Carter', 'Alexander', 
                       'Maisie Dunlop', 'Meekin', 'Nance Maguire', 'Hathercleugh', #perp
                       'Dunlop', 'John Paley', 'Robertson', 'Tom Dunlop', 
                       'Hugh Moneylaws', 'Scott', 'Watson', 'Martin Smeaton', 
                       'Turndale']
    }
}

In [ ]:
# Causes output text to wrap in output cells
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
Match = namedtuple('Match', ['chapter_number', 'sentence_number', 'matches', 'text'])


class Scraper:
    def __init__(self, urls):
        self.urls = urls

    def scrape(self, *, responses=None):
        books = []
        responses = [requests.get(url) for url in self.urls] if responses is None else responses

        for response in responses:

            positions = [
                         response.text.find("***END"),
                         response.text.find("THE END")
                        ]
            end_pos = min([pos for pos in positions if pos > 0])
            book_html = response.text[:end_pos]

            def search(pattern):
                return re.search(pattern, book_html).group(1)

            title = search('Title: ([^\<]*)')
            author = search('Author: ([^\<]*)')
            release = search('Release Date: ([A-Za-z]*[\s0-9]*?, [0-9]*)')

            soup = BeautifulSoup(book_html)

            chapters = self.find_chapters(soup)

            books.append(Book(author, chapters, release, title))

        return books

    def find_chapters(self, soup):
        chapters = []
        chapter_tags = soup.find_all(
            re.compile("(h2|h3)"),
            string=re.compile("CHAPTER")
        )
        for chapter_number, chapter_tag in enumerate(chapter_tags, start=1):
            chapter_text = []
            chapter_title = None
            for i, sibling in enumerate(chapter_tag.next_siblings):
                if i == 1:
                    chapter_title = string.capwords(sibling.text)
                elif sibling.name == 'p':
                    chapter_text.append(sibling.text)
                elif sibling.name == 'h2':
                    break
                else:
                    continue

            chapters.append(
                Chapter(chapter_title, chapter_number, chapter_text)
            )

        return chapters

class Chapter:
    def __init__(self, title, number, text):
        self.title = title
        self.number = number
        self.text = self.clean(text)

    def find(self, pattern):
        all_matches = []
        for sentence_number, sentence in self.sentences:
            matches = list(re.finditer(pattern, sentence))
            if len(matches):
                all_matches.append(
                    Match(self.number, sentence_number, matches, sentence)
                )

        return all_matches

    @staticmethod
    def clean(chapter_text):
        chapter_text = ' '.join(chapter_text)
        chapter_text = chapter_text.replace('\r\n', ' ')
        return chapter_text

    @property
    def sentences(self):
        '''
        Splits the chapter text into sentences. This is a hard task so some decisions need to be made.

        Case 1: (?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<![A-Z][a-z][a-z]\.)(?<=[.?!])\s
            (?<!\w\.\w.): Negative lookbehind to prevent splitting on i.e. and e.g.
        Python requires fixed width patterns for lookbehinds, so we have split these
            (?<![A-Z][a-z]\.): Negative lookbehind to prevent splitting on Mr.
            (?<![A-Z][a-z][a-z]\.): Negative lookbehind to prevent splitting on Mrs.
            (?<=[.?!]): Positive lookbehind to make sure we're only splitting after ., ?, or !
            \s: Any white-space character
        Case 2: (?<=[.?!][\"])\s(?=[\"A-Z])
            (?<=[.?!][\"]): Positive lookbehind to make sure we're only splitting after .", ?", or !"
            \s: Any white-space character
            (?=[\"A-Z]): Positive lookahead to make sure we're only splitting before " or a capital letter

        Examples:
            ...a fashion which had become a habit. Miss Penkridge...
                Sentence 1: ...a fashion which had become a habit.
                Sentence 2: Miss Penkridge...
            ...content. "So he did it! Now, I should never have thought it! The last person...
                Sentence 1: ...content.
                Sentence 2: "So he did it!
                Sentence 3: Now, I should never have thought it!
                Sentence 4: The last person...
            ...this sort of stuff?" "Stuff?" demanded Miss Penkridge, who had resumed her...
                Sentence 1: ...this sort of stuff?"
                Sentence 2: "Stuff?" demanded Miss Penkridge, who had resumed her...
        '''
        splits = re.split('(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<![A-Z][a-z][a-z]\.)(?<=[.?!])\s|(?<=[.?!][\"])\s(?=[\"A-Z])', self.text)
        return zip(range(1, len(splits)+1), splits)

    def __len__(self):
        return len(list(self.sentences))

    def __str__(self):
        return self.title

    def __repr__(self):
        return (
            f'Chapter {self.number}: {self.title}\n'
            f'Content: {self.text[:20]}...{self.text[-20:]}'
        )

class Book:
    def __init__(self, author, chapters, release, title):
        self.author = author
        self.chapters = chapters
        self.release = release
        self.title = title

    def find(self, pattern):
        all_matches = []
        for chapter in self.chapters:
            matches = chapter.find(pattern)
            if len(matches):
                all_matches.extend(matches)

        return all_matches

    def __len__(self):
        return len(self.chapters)

    def __str__(self):
        return self.title

    def __repr__(self):
        return (
            f'{self.title} by {self.author}, '
            f'released in {self.release} and '
            f'contains {len(self)} chapters.'
        )

In [ ]:
# For debugging
responses = [requests.get(url) for url in urls]

In [ ]:
scraper = Scraper(urls)
books = scraper.scrape(responses=responses)

In [ ]:
from collections import Counter

detail_to_readable = {
    'Detective': 'Detective',
    'Crime': 'Crime',
    'Perp': 'Perpetrator',
    'Other Suspects': 'Other Suspects'
}

words_before = 3
words_after = 3
stats = []
for book in books:
    for detail in book_details[book.title]:
        if detail in ['Detective', 'Crime', 'Perp', 'Other Suspects']:
            pattern = book_details[book.title][detail]
            raw_results = [(m.chapter_number, m.sentence_number) for m in book.find(f'(?i){pattern}')]
            counter = Counter({i: 0 for i in range(1, len(book)+1)})
            counter.update([result[0] for result in raw_results])
            book_stats = {
                'title': book.title,
                'author': 'J.S. Fletcher',
                'queryType': 'occurrences',
                'query': pattern,
                'question': detail_to_readable[detail],
                'numChapters': len(book),
                'results': dict(counter)
            }
            stats.append(book_stats)
        if detail == 'Detective':
            pattern = (
                "(\w+)[^\w]*\s" * words_before +
                book_details[book.title][detail] +
                "\s[^\w]*(\w+)" * words_after
            )
            matches = book.find(pattern)
            before_counters = [Counter() for _ in range(words_before)]
            after_counters = [Counter() for _ in range(words_after)]
            for match in matches:
                for regex_match in match.matches:
                    near_words = regex_match.groups()
                    for i, word in enumerate(near_words):
                        if word is None:
                            continue
                        if i < words_before:
                            before_counters[i].update({ word })
                        else:
                            after_counters[i - words_before].update({ word })

            after = [('', 0)] * len(after_counters)
            for i, counter in enumerate(after_counters):
                for word in counter.most_common(1):
                    after[i] = word

            book_stats = {
                'title': book.title,
                'author': 'J.S. Fletcher',
                'queryType': 'nearby',
                'query': book_details[book.title][detail],
                'question': 'Nearby Words',
                'numChapters': len(book),
                'before': [word for counter in before_counters for word in counter.most_common(1)],
                'after': after
            }
            stats.append(book_stats)



print(stats)

import json
with open('/gdrive/My Drive/fletcher.json', 'w') as f:
    json.dump(stats, f)

[{'title': 'The Middle of Things', 'author': 'J.S. Fletcher', 'queryType': 'occurrences', 'query': 'Viner', 'question': 'Detective', 'numChapters': 29, 'results': {1: 23, 2: 41, 3: 28, 4: 41, 5: 35, 6: 25, 7: 17, 8: 26, 9: 11, 10: 26, 11: 10, 12: 35, 13: 31, 14: 29, 15: 34, 16: 30, 17: 10, 18: 12, 19: 5, 20: 12, 21: 10, 22: 12, 23: 3, 24: 12, 25: 45, 26: 9, 27: 36, 28: 44, 29: 18}}, {'title': 'The Middle of Things', 'author': 'J.S. Fletcher', 'queryType': 'nearby', 'query': 'Viner', 'question': 'Nearby Words', 'numChapters': 29, 'before': [('the', 8), ('he', 6), ('and', 19)], 'after': [('had', 13), ('a', 14), ('to', 12)]}, {'title': 'The Middle of Things', 'author': 'J.S. Fletcher', 'queryType': 'occurrences', 'query': 'Cortelyon', 'question': 'Perpetrator', 'numChapters': 29, 'results': {1: 0, 2: 4, 3: 2, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 16, 29: 8}}, {'t

In [ ]:


patterns = ['Ashton', 'Stafford', 'Pratt', 'Driscoll', 'Gaffney']

stats = []
for pattern, book in zip(patterns, books):
    book_stats = {'title': book.title}
    book_stats['chapter_lengths'] = [len(chapter) for chapter in book.chapters]
    book_stats['query_type'] = 'Occurences'
    book_stats['query'] = pattern
    book_stats['results'] = [(m.chapter_number, m.sentence_number) for m in book.find(pattern)]
    stats.append(book_stats)

with open('/gdrive/My Drive/test.json', 'w') as f:
    json.dump(stats, f)

In [ ]:
scraper = Scraper(urls)
books = scraper.scrape(responses=responses[:1])

for book in books:
    print(f"Book: {book.title}, {len(book)} Chapters")
    for chapter in book.chapters:
        nl = '\n'
        print(f"    Chapter: {chapter.title}")
        print(f"        {len(chapter)} sentences")
        for sent in chapter.sentences:
            print(sent[1])
        break

Book: The Middle of Things, 29 Chapters
    Chapter: Faced With Reality
        155 sentences
On that particular November evening, Viner, a young gentleman of means and leisure, who lived in a comfortable old house in Markendale Square, Bayswater, in company with his maiden aunt Miss Bethia Penkridge, had spent his after-dinner hours in a fashion which had become a habit.
Miss Penkridge, a model housekeeper and an essentially worthy woman, whose whole day was given to supervising somebody or something, had an insatiable appetite for fiction, and loved nothing so much as that her nephew should read a novel to her after the two glasses of port which she allowed herself every night had been thoughtfully consumed and he and she had adjourned from the dining-room to the hearthrug in the library.
Her tastes, however, in Viner's opinion were somewhat, if not decidedly, limited.
Brought up in her youth on Miss Braddon, Wilkie Collins and Mrs.
Henry Wood, Miss Penkridge had become a confirmed s

In [ ]:
scraper = Scraper(urls)
books = scraper.scrape(responses=responses)

all_matches = books[0].find("Ashton")
for match in all_matches:
    print(f'Found a match in Chapter {match.chapter_number}, Sentence {match.sentence_number}')
    print(f'Sentence Text: {match.text}')
    print(match.matches[0].group(0))
    break

Found a match in Chapter 2, Sentence 23
Sentence Text: "Mr. Ashton!" "You know him?" said Viner.
Ashton


In [ ]:
print(books)

[The Middle of Things by J. S. Fletcher, released in February, 2006 and contains 29 chapters., Scarhaven Keep by J. S. Fletcher, released in February, 2006 and contains 31 chapters., The Talleyrand Maxim by J. S. Fletcher, released in February, 2006 and contains 28 chapters., The Middle Temple Murder by J.S. Fletcher, released in December 3, 2003 and contains 36 chapters., The Rayner-Slade Amalgamation by J. S. Fletcher, released in December 12, 2003 and contains 35 chapters.]
